## Trajectory extraction from Rosbag file



The goal of this script is to take a raw rosbag recording of a scene and partition out a set of discrete throw demonstrations.

In [2]:
import numpy as np
import pandas as pd

## Position interpolation, shared argument axis

The goal of this section is producing time series data with all the relevant parameter values by interpolation.

## Gripper separation event detection

The goal of this section is to produce a timestamp 